In [4]:
%matplotlib qt

In [1]:

import genseq
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D


T= V = H = 50

seqt = genseq.genseq_t(T, V, H, 1, 1)
seqv = genseq.genseq_v(T, V, H, 1, 1)
seqh = genseq.genseq_h(T, V, H, 1, 1)

seqtv = genseq.genseq_tv(T, V, H, 1, 1)
seqth = genseq.genseq_th(T, V, H, 1, 1)
seqvh = genseq.genseq_vh(T, V, H, 1, 1)

seqtvh = genseq.genseq_tvh(T, V, H, 1, 1)

seq_tot = seqt + seqv + seqh + seqtv + seqth + seqvh+ seqtvh

v_min = np.min(seq_tot)
v_max = np.max(seq_tot)

list_seq = [seqt, seqtv, seqth, seqtvh, seq_tot, seqv, seqh, seqvh]
title_list = ["$n_t$", "$n_{tv}$", "$n_{th}$", "$n_{tvh}$", "$n_{tot}$", "$n_{v}$", "$n_{h}$", "$n_{vh}$"]

# Parcourir les séquences

In [6]:
import matplotlib.pyplot as plt


#plt.rcParams['keymap.<command>'] = ['<key1>', '<key2>']

def remove_keymap_conflicts(new_keys_set):
    for prop in plt.rcParams:
        if prop.startswith('keymap.'):
            keys = plt.rcParams[prop]
            remove_list = set(keys) & new_keys_set
            for key in remove_list:
                keys.remove(key)


#fig, ax = plt.subplots(nrows=4, ncols=2)

#plt.imshow(data[0])



def multi_slice_viewer(volumes, title_list):
    remove_keymap_conflicts({'j', 'k'})
    fig, ax = plt.subplots(nrows=2, ncols=4, squeeze=True)
    i=0
    for myax in ax.flatten():
        #myax = ax[i
        myax.volume = volumes[i]
        myax.index = volumes[i].shape[0] // 2
        myax.imshow(volumes[i][myax.index], vmin=v_min, vmax=v_max, interpolation=None, cmap="gray")
        myax.set_title(title_list[i])
        myax.axis("off")
        fig.canvas.mpl_connect('key_press_event', process_key)
        i+=1

def process_key(event):
    fig = event.canvas.figure
    for i in [0, 1, 2, 3, 4, 5, 6, 7]:
        ax = fig.axes[i]
        if event.key == 'j':
            previous_slice(ax)
        elif event.key == 'k':
            next_slice(ax)
        fig.canvas.draw()

def previous_slice(ax):
    volume = ax.volume
    ax.index = (ax.index - 1) % volume.shape[0]  # wrap around using %
    ax.images[0].set_array(volume[ax.index])

def next_slice(ax):
    volume = ax.volume
    ax.index = (ax.index + 1) % volume.shape[0]
    ax.images[0].set_array(volume[ax.index])
    

#plt.imshow(data[0])

multi_slice_viewer(list_seq, title_list)


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


N = 4
volume = np.random.rand(N, N, N)

def display3dseq(seq):
    T, V, H = seq.shape
    filled = np.ones((T, V, H), dtype=np.bool)
    colors = np.repeat(seq[:, :, :, np.newaxis], 3, axis=3)
    
    fig = plt.figure()
    
    ax = fig.gca(projection='3d')
    ax.voxels(filled, facecolors=colors, edgecolors='k')

    plt.show()

In [10]:
display3dseq(volume)

In [11]:
%matplotlib qt

import numpy as np


def display8seq(vec_seq, titles=False):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    T, V, H = vec_seq[0].shape
    filled_list = [np.ones((T, V, H), dtype=np.bool) for k in range(len(vec_seq))]
    
    v_min = np.min(vec_seq)
    v_max = np.max(vec_seq)
    colors = (vec_seq-v_min)/(v_max-v_min)
    #print(colors.shape)
    colors_list = [(np.repeat(seq[:, :, :, np.newaxis], 3, axis=3)-v_min)/(v_max-v_min) for seq in vec_seq]
    #print(colors_list)
    fig, axes_arr = plt.subplots(2, 4, subplot_kw=dict(projection='3d'))
    axes = axes_arr.flatten()
    #ax = fig.gca(projection='3d')
    for ax, i in zip(axes, range(len(vec_seq))):
        ax.voxels(filled_list[i], facecolors=colors_list[i])#, edgecolors='k')
        ax.set_xlabel('T axis')
        ax.set_ylabel('V axis')
        ax.set_zlabel('H axis')
        ax.set_xticks([]) 
        ax.set_yticks([]) 
        ax.set_zticks([])
        #plt.tight_layout()

        if titles:
            ax.set_title(titles[i])


In [12]:
display8seq(list_seq, title_list)



In [51]:
import noise

def fast_display_all_noises(seq, pc=False):
    T, V, H = seq.shape
    mu= np.mean(seq)
    noises = noise.get_all_3d_noise_var_fast(seq)
    seq_noises = noises[0]
    stot = noises[1]
    st, sv, sh, stv, sth, svh, stvh = seq_noises
    
    print("-------------------------------------------------")
    print("Mean : {:.3f} | Sigma : {:.3f}".format(mu, stot))
    print("T : {} | V : {} |  H : {} | Max error : {:.1%}".format(T, V, H, ((1-1/T-1/V-1/H)-1)))
    print("-------------------------------------------------")
    if pc: 
        st_pc, sv_pc, sh_pc, stv_pc, sth_pc, svh_pc, stvh_pc = tuple(np.array(seq_noises)/stot)
        print('$\sigma_t$ : {:8.3%} | $\sigma_tv$ : {:6.3%} | $\sigma_th$ : {:6.3%} | $\sigma_tvh$ : {:6.3%} '.format(st_pc, stv_pc, sth_pc, stvh_pc))
        print('$\sigma_tot$ : {:6.3f} | $\sigma_v$ :  {:6.3%} | $\sigma_h$ :  {:6.3%} | $\sigma_vh$ :  {:6.3%}'.format(stot, sv_pc, sh_pc, svh_pc))
    else:
        print('$\sigma_t$ : {:8.3f} | $\sigma_tv$ : {:6.3f} | $\sigma_th$ : {:6.3f} | $\sigma_tvh$ : {:6.3f} '.format(st, stv, sth, stvh))
        print('$\sigma_tot$ : {:6.3f} | $\sigma_v$ :  {:6.3f} | $\sigma_h$ :  {:6.3f} | $\sigma_vh$ :  {:6.3f}'.format(stot, sv, sh, svh))
    print("-------------------------------------------------")

    
fast_display_all_noises(seq_tot, pc=True)

-------------------------------------------------
Mean : 7.082 | Sigma : 7.084
T : 50 | V : 50 |  H : 50 | Max error : -6.0%
-------------------------------------------------
$\sigma_t$ :  16.693% | $\sigma_tv$ : 14.043% | $\sigma_th$ : 14.129% | $\sigma_tvh$ : 13.305% 
$\sigma_tot$ :  7.084 | $\sigma_v$ :  14.703% | $\sigma_h$ :  13.203% | $\sigma_vh$ :  13.925%
-------------------------------------------------


In [52]:
a, b, c = (1, 2, 3)

In [32]:
print(a)
print(b)
print(c)

1
2
3


In [37]:
a = noise.get_all_3d_noise_var_fast(seq_tot)[0]
a

(1.1824731397768695,
 1.0415200269876437,
 0.9352666193700954,
 0.9947348799788408,
 1.0008279418155794,
 0.9863708135217796,
 0.942491042135825)

In [38]:
s, t, c, v, g, h, j = a

In [39]:
s

1.1824731397768695

In [40]:
t

1.0415200269876437